In [2]:
!pip install sseclient


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [27]:
import json
import threading
import time
from abc import ABC, abstractmethod
from dataclasses import dataclass
from functools import cached_property
from threading import Thread
from typing import Any, Callable, Dict, List, Literal, Optional

import requests
from sseclient import SSEClient

In [28]:
USERNAME = "Junrong"
PASSWORD = "ads100"

In [29]:
STANDARD_HEADERS = {"Content-Type": "application/json; charset=utf-8"}


def _get_headers(auth_token: str) -> Dict[str, str]:
    return {**STANDARD_HEADERS, "Authorization": auth_token}


class SSEThread(Thread):
    bearer: str
    url: str
    handler_method: Callable[[Dict], Any]

    def __init__(self, bearer: str, url: str, handler_method: Callable[[Dict], Any]):
        super().__init__()
        self.bearer = bearer
        self.url = url
        self.handler_method = handler_method

    def run(self):
        while True:
            try:
                self._start_sse_client()
            except:
                pass

    def _start_sse_client(self):
        headers = {
            "Authorization": self.bearer,
            "Accept": "text/event-stream; charset=utf-8",
        }
        messages = SSEClient(self.url, headers=headers)
        for msg in messages:
            if msg.id is not None:
                self.handler_method(json.loads(msg.event))

# Basic Example bot

In [30]:
@dataclass
class BasicCMIBot:
    username: str
    password: str
    _cmi_url: str = "http://cmi-eu-2"

    @cached_property
    def auth_token(self):
        return self._authenticate()

    def send_order(
        self, product: str, price: float, side: Literal["BUY", "SELL"], volume: int
    ) -> Dict:
        payload = {"product": product, "side": side, "price": price, "volume": volume}
        url = f"{self._cmi_url}/api/order"
        response = requests.post(
            url, json=payload, headers=_get_headers(self.auth_token),
            verify=False
        )
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to send order, {payload}")
            print(response.json())

    def get_all_orders(self) -> Optional[List[Dict]]:
        url = f"{self._cmi_url}/api/order/current-user"
        response = requests.get(url, headers=_get_headers(self.auth_token), verify=False)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to get all orders: {response.json()}")

    def cancel_order(self, product, price) -> Dict:
        url = f"{self._cmi_url}/api/order?product={product}&price={price}"
        response = requests.delete(url, headers=_get_headers(self.auth_token), verify=False)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to delete order: {response.json()}")

    def cancel_all_orders(self):
        for order in self.get_all_orders():
            url = f'{self._cmi_url}/api/order/{order["id"]}'
            response = requests.delete(url, headers=_get_headers(self.auth_token), verify=False)
            if response.status_code != 200:
                print("Failed to cancel order")
                print(response.json())

    def get_all_products(self) -> List[str]:
        url = f"{self._cmi_url}/api/product"
        response = requests.get(url, headers=STANDARD_HEADERS, verify=False)
        if response.status_code == 200:
            return [product["symbol"] for product in json.loads(response.text)]
        else:
            print("Failed to get all products")
            print(response.json())

    @property
    def positions(self) -> Dict[str, int]:
        url = f"{self._cmi_url}/api/position/current-user"
        response = requests.get(url, headers=_get_headers(self.auth_token), verify=False)
        if response.status_code == 200:
            return {
                position["product"]: position["volume"] for position in response.json()
            }
        else:
            print("Failed to get positions")
            print(response.json())
            return dict()

    def _authenticate(self) -> str:
        auth = {"username": self.username, "password": self.password}
        url = f"{self._cmi_url}/api/user/authenticate"
        response = requests.post(url, headers=STANDARD_HEADERS, json=auth, verify=False)
        response.raise_for_status()
        return response.headers["Authorization"]

In [31]:
basic_bot = BasicCMIBot(username=USERNAME, password=PASSWORD)

In [32]:
basic_bot.get_all_products()

/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['150 CALL', 'FUTURE', '130 PUT']

In [33]:
from math import ceil, floor


def round_down_to_tick(price: float, tick_size: float = 0.1):
    """
    Rounds a price down to the nearest tick, e.g. if the tick size is 0.10, a price of 0.97 will get rounded to 0.90.
    """
    return floor(price / tick_size) * tick_size


def round_up_to_tick(price: float, tick_size: float = 0.1):
    """
    Rounds a price up to the nearest tick, e.g. if the tick size is 0.10, a price of 1.34 will get rounded to 1.40.
    """
    return ceil(price / tick_size) * tick_size

theo = 140
cnt = 20.0
while True:
    price = int(input())
    theo += (price - 7)
    cnt -= 1

    basic_bot.send_order(product="FUTURE", price=theo-0.5, side="BUY", volume=250)
    basic_bot.send_order(product="FUTURE", price=theo+0.5, side="SELL", volume=250)
    basic_bot.cancel_all_orders()
    # call_theo = max( 0, (theo - 150 + cnt / 5))
    # put_theo = max (0, (130 - theo + cnt / 5))    
    # basic_bot.send_order(product="150 CALL", price=round_down_to_tick(call_theo-0.25, 0.25), side="BUY", volume=250)
    # basic_bot.send_order(product="150 CALL", price=round_up_to_tick(theo+0.25, 0.25), side="SELL", volume=250)
    # basic_bot.send_order(product="130 PUT", price=round_down_to_tick(theo-0.25, 0.25), side="SELL", volume=250)
    # basic_bot.send_order(product="130 PUT", price=round_up_to_tick(theo+0.25, 0.25), side="SELL", volume=250)
    


/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.1

Failed to send order, {'product': 'FUTURE', 'side': 'SELL', 'price': 138.5, 'volume': 250}
{'errorCode': 'BAD_REQUEST', 'message': 'Volume of 250 specified would exceed the exposure limit. Canceled.', 'timestamp': '2024-07-26T10:21:31.164290766Z', 'requestUri': '/api/order'}


/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.1

Failed to send order, {'product': 'FUTURE', 'side': 'SELL', 'price': 134.5, 'volume': 250}
{'errorCode': 'BAD_REQUEST', 'message': 'Volume of 250 specified would exceed the exposure limit. Canceled.', 'timestamp': '2024-07-26T10:21:37.425755368Z', 'requestUri': '/api/order'}


/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.1

Failed to send order, {'product': 'FUTURE', 'side': 'SELL', 'price': 128.5, 'volume': 250}
{'errorCode': 'BAD_REQUEST', 'message': 'Volume of 250 specified would exceed the exposure limit. Canceled.', 'timestamp': '2024-07-26T10:23:26.676485709Z', 'requestUri': '/api/order'}


/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Failed to cancel order


/Users/jchen/Code/mm/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmi-eu-2'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Bot That Subscribes to the Exchange

In [10]:
@dataclass
class SubscribableCMIBotBase(BasicCMIBot, ABC):
    @abstractmethod
    def on_orderbook(self, orderbook):
        raise NotImplementedError("You must implement the on_orderbook method!")

    @abstractmethod
    def on_trade(self, trade):
        raise NotImplementedError("You must implement the on_trade method!")

    def __post_init__(self):
        self.products = self.get_all_products()
        for product in self.products:
            product_url = f"{self._cmi_url}/api/order/{product}/stream"
            product_thread = SSEThread(
                bearer=self.auth_token,
                url=product_url,
                handler_method=self._on_orderbook_wrapper,
            )
            product_thread.start()

        trades_url = f"{self._cmi_url}/api/trade/stream"
        trades_thread = SSEThread(
            bearer=self.auth_token, url=trades_url, handler_method=self.on_trade
        )
        trades_thread.start()

    def _on_orderbook_wrapper(self, orderbook: Dict[str, Any]):
        orderbook["product"] = orderbook.pop("productsymbol")
        orderbook["buy_orders"] = sorted(
            [
                {
                    "price": float(price),
                    "volume": volumes["marketVolume"],
                    "own_volume": volumes["userVolume"],
                }
                for price, volumes in orderbook["buyOrders"].items()
            ],
            key=lambda d: -d["price"],
        )
        orderbook["sell_orders"] = sorted(
            [
                {
                    "price": float(price),
                    "volume": int(volumes["marketVolume"]),
                    "own_volume": int(volumes["userVolume"]),
                }
                for price, volumes in orderbook["sellOrders"].items()
            ],
            key=lambda d: d["price"],
        )
        del orderbook["buyOrders"]
        del orderbook["sellOrders"]
        del orderbook["midPrice"]
        self.on_orderbook(orderbook)

    def keep_alive(self):
        while True:
            time.sleep(5)

In [11]:
class ExampleBot(SubscribableCMIBotBase):

    def on_orderbook(self, orderbook):
        print(orderbook)

    def on_trade(self, trade):
        print(trade)

In [12]:
smart_bot = ExampleBot(username=USERNAME, password=PASSWORD)

bot_thread = threading.Thread(target=smart_bot.keep_alive)
bot_thread.daemon = True
bot_thread.start()

In [13]:
basic_bot.send_order(product="example", price=20, side="BUY", volume=30)

{'tickSize': 1, 'product': 'example', 'buy_orders': [{'price': 20.0, 'volume': 360.0, 'own_volume': 360.0}], 'sell_orders': []}


{'id': 'ac4c5174-45b8-4e08-9079-5a4fe1e1a846',
 'status': 'ACTIVE',
 'product': 'example',
 'side': 'BUY',
 'price': 20,
 'volume': 30,
 'filled': 0,
 'message': '',
 'user': 'test',
 'timestamp': '2024-07-26T05:04:27.222098775Z'}